In [1]:
import glob
import os
import nibabel as nib
import numpy as np
from scipy.ndimage.interpolation import zoom
from scipy import ndimage
import pickle
import nibabel as nib
import random
from scipy.ndimage.interpolation import zoom
from tqdm import tqdm
import matplotlib.pyplot as plt

from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchcontrib.optim import SWA


In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def read_mutant_txt(path):
    name_list = []
    fo = open(path)
    for line in fo:
        striped_line = line.strip('\n')
        if striped_line != '':
            name_list.append(striped_line)
    return name_list


In [3]:
mutant_names = read_mutant_txt('mutant_imgs.txt')
data_base_path = '/scratch/zq415/grammar_cor/mutant_detect/mutant_cla/data'
data_nii_path = '/scratch/zq415/grammar_cor/mutant_detect/mutant_cla/bv_body_nii'
save_name = 'All_data_256_196_160_down.pickle'#'All_data_256_196_160.pickle'


In [4]:
if os.path.exists(os.path.join(data_base_path,save_name)):
    with open(os.path.join(data_base_path, save_name), "rb") as input_file:
        all_train_data = pickle.load(input_file)
        
else:
    files = glob.glob(data_nii_path + '/*.nii')
    all_train_data = {}
    for i in tqdm(range(len(files))):
        base_name = os.path.basename(files[i])[:-17]
        label = nib.load(files[i])
        ori_label = np.uint8(label.get_data()[::2,::2,::2])
        print(ori_label.shape)
        all_train_data[base_name] = ori_label
    if not os.path.exists(os.path.join(data_base_path,save_name)):
        save_file = open(os.path.join(data_base_path, save_name),'wb')
        pickle.dump(all_train_data,save_file)
        save_file.close()
        
        

In [5]:
mutant_group = [(9,10), (12,13,14), (16,17,18,19), (36,37), (38,39), (42,43), (44,45,46,47), (48,49,50,51), (52,53), (54,55),
(56,57,58), (59,60,61), (63,64), (66,67,68), (69,70,71), (73,74), (75,76), (77,78,79), (80,81,82,83,84,85,86,87),
(89,90), (91,92), (93,94), (95,96,97), (98,99), (100,101,102)]

group_list = []
for one_group in mutant_group:
    for ii in range(len(one_group)):
        group_list.append(one_group[ii])
single_mutant = [i for i in range(len(mutant_names)) if i not in group_list]

test_mut_names = []
for i in range(2,len(mutant_group),6):
    for ii in range(len(mutant_group[i])):
        test_mut_names.append(mutant_names[mutant_group[i][ii]])

for i in range(2,len(single_mutant),6):
    test_mut_names.append(mutant_names[single_mutant[i]])
    
print(len(test_mut_names),test_mut_names)


17 ['20170207_En1_E13_E12a_Mut', '20170207_En1_E13_E12b_Mut', '20170207_En1_E13_E12c_Mut', '20170207_En1_E13_E12d_Mut', '20170727_En1_E13_E1a_Mut', '20170727_En1_E13_E1b_Mut', '20161222_En1_E13_E1a_Mut', '20161222_En1_E13_E1b_Mut', '20161222_En1_E13_E1c_Mut', '20180202_En1_E14_E4a_mut', '20180202_En1_E14_E4b_Mut', '20171211_En1_E10_E4b', '20170705_En1_E12_E1a_filt', '20171004_En1_E13_E5b_Mut', '20171009_En1_E12_E11a', '20161219_En1_E10_E5Ma', '20161223_En1_E14_E1a_Mut']


In [6]:
# #for bv only
# for i_key in all_train_data:
#     all_train_data[i_key] = np.uint8(all_train_data[i_key] > 1.5)
    
# train_data = []
# test_data = []
# seed_num = 0

# for name in all_train_data:
#     if name in mutant_names:
#         if name in test_mut_names:
#             test_data.append((all_train_data[name]-0.5, 0))
#             print('mutant: ',name)
#         else:
#             train_data.append((all_train_data[name]-0.5, 0))
#     else:
#         random.seed(seed_num*8)
# #         random.seed(seed_num*9)
#         seed_num += 1
#         if random.uniform(0,1) < 0.16:
#             test_data.append((all_train_data[name]-0.5, 1))
#             print('normal: ',name)
#         else:
#             train_data.append((all_train_data[name]-0.5, 1))

# print(len(test_data))
# print(len(train_data))

In [7]:
# #for body only
# for i_key in all_train_data:
#     all_train_data[i_key] = np.uint8(all_train_data[i_key] > 0.5)
    
# train_data = []
# test_data = []
# seed_num = 0

# for name in all_train_data:
#     if name in mutant_names:
#         if name in test_mut_names:
#             test_data.append((all_train_data[name]-0.5, 0))
#             print('mutant: ',name)
#         else:
#             train_data.append((all_train_data[name]-0.5, 0))
#     else:
#         random.seed(seed_num*8)
# #         random.seed(seed_num*9)
#         seed_num += 1
#         if random.uniform(0,1) < 0.16:
#             test_data.append((all_train_data[name]-0.5, 1))
#             print('normal: ',name)
#         else:
#             train_data.append((all_train_data[name]-0.5, 1))

# print(len(test_data))
# print(len(train_data))

In [6]:
mutant_num = 0
normal_num = 0

for name in all_train_data:
    if name in mutant_names:
        mutant_num += 1
    else:
        normal_num += 1
        
print(mutant_num, normal_num)

102 440


In [8]:
#####for bv and body
train_data = []
test_data = []
seed_num = 0

for name in all_train_data:
    if name in mutant_names:
        if name in test_mut_names:
            test_data.append((all_train_data[name]-1.0, 0))
            print('mutant: ',name)
        else:
            train_data.append((all_train_data[name]-1.0,0))
    else:
        random.seed(seed_num*8)
#         random.seed(seed_num*9)
        seed_num += 1
        if random.uniform(0,1) < 0.16:
            test_data.append((all_train_data[name]-1.0,1))
            print('normal: ',name)
        else:
            train_data.append((all_train_data[name]-1.0,1))

print(len(test_data))
print(len(train_data))


normal:  20180228_En1_E13p5_Ext_E1a
normal:  20180302_En1_E14.5_Ext_E11a_reg
normal:  20171212_En1_E11_E8a
normal:  20180202_En1_E14_E12a
normal:  20171003_En1_E13_E8a_reg
normal:  20170129_En1_E11_E2a_reg
normal:  20170206_En1_E12_E3a
normal:  20170320_En1_E11_E6a
normal:  20180205_En1_E11_E1c_reg
mutant:  20180202_En1_E14_E4a_mut
normal:  20180302_En1_E14.5_Ext_E8a_reg
normal:  20180904_En1_E11p5_4a_007
mutant:  20170727_En1_E13_E1a_Mut
normal:  20170313_EN1_E10_E3b_reg_filt
normal:  20161207_En1_E14_E4a
normal:  20170619_En1_E12_E2a
normal:  20180202_En1_E14_E1a
normal:  20180226_En1_E11p5_Em2a
normal:  20161116_En1_E12_E6a_reg
normal:  20161222_En1_E13_E2a
normal:  20171002_En1_E11_E1a
normal:  20170315_En1_E12_E2b
mutant:  20161222_En1_E13_E1c_Mut
normal:  20170331_En1_E14p_E2a
normal:  20170705_En1_E12_E5a_filt
normal:  20161116_En1_E12_E2a_reg
normal:  20170619_En1_E12_Ex_E12a_reg
normal:  20180226_En1_E11p5_Em1a
mutant:  20170207_En1_E13_E12a_Mut
normal:  20170314_En1_E11_E4b
m

In [9]:
#img_nft = nib.Nifti1Image(np.squeeze(all_train_data[193]),np.eye(4))
#img_save_data_path = './img/mul_img.nii'
#nib.save(img_nft,img_save_data_path)

In [10]:
from torch.utils.data import Dataset, DataLoader

class Mouse_sub_volumes(Dataset):
    """Mouse sub-volumes BV dataset."""

    def __init__(self, all_data , transform=None):
        """
        Args:
            all_whole_volumes: Contain all the padded whole BV volumes as a dic
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.all_data = all_data
        self.transform = transform
    def __len__(self):
        return len(self.all_data)

    def __getitem__(self, num):
        
        current_img, label = self.all_data[num]
        
        img = np.float32(current_img[np.newaxis,...])
        sample = {'image': img, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [11]:
class Flip(object):
    
    """
    Flip the image for data augmentation, but prefer original image.
    """
    
    def __init__(self,ori_probability=0.20):
        self.ori_probability = ori_probability

    def __call__(self, sample):
        if random.uniform(0,1) < self.ori_probability:
            return sample
        else:
            img, label = sample['image'], sample['label']
            random_choise1=random.choice([1,2,3,4,5,6,7,8])
            img[0,...] = {1: lambda x: x,
                          2: lambda x: x[::-1,:,:],
                          3: lambda x: x[:,::-1,:],
                          4: lambda x: x[:,:,::-1],
                          5: lambda x: x[::-1,::-1,:],
                          6: lambda x: x[::-1,:,::-1],
                          7: lambda x: x[:,::-1,::-1],
                          8: lambda x: x[::-1,::-1,::-1]
                          }[random_choise1](img[0,...])
        return {'image': img, 'label': label}

In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class VGG_net(nn.Module):
    def __init__(self,conv_drop_rate=0.10,linear_drop_rate=0.10):
        super(VGG_net, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=12, kernel_size=3,stride=1, padding=2,dilation=2)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1_bn = nn.BatchNorm3d(12)
        self.conv2 = nn.Conv3d(in_channels=12, out_channels=12, kernel_size=3,stride=1,padding=2, dilation=2)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2_bn = nn.BatchNorm3d(12)
        self.pool1 = nn.MaxPool3d(2, 2)
        self.dropout1 = nn.Dropout3d(conv_drop_rate)
        
        self.conv3 = nn.Conv3d(in_channels=12, out_channels=24, kernel_size=3,stride=1, padding=2,dilation=2)
        self.relu3 = nn.ReLU(inplace=True)
        self.conv3_bn = nn.BatchNorm3d(24)
        self.conv4 = nn.Conv3d(in_channels=24, out_channels=24, kernel_size=3,stride=1, padding=2,dilation=2)
        self.relu4 = nn.ReLU(inplace=True)
        self.conv4_bn = nn.BatchNorm3d(24)
        self.pool2 = nn.MaxPool3d(2, 2)
        self.dropout2 = nn.Dropout3d(conv_drop_rate)
        
        self.conv5 = nn.Conv3d(in_channels=24, out_channels=48, kernel_size=3,stride=1, padding=2,dilation=2)
        self.relu5 = nn.ReLU(inplace=True)
        self.conv5_bn = nn.BatchNorm3d(48)
        self.conv6 = nn.Conv3d(in_channels=48, out_channels=48, kernel_size=3,stride=1, padding=2,dilation=2)
        self.relu6 = nn.ReLU(inplace=True)
        self.conv6_bn = nn.BatchNorm3d(48)
        self.pool3 = nn.MaxPool3d(2, 2)
        self.dropout3 = nn.Dropout3d(conv_drop_rate)
        
        self.conv7 = nn.Conv3d(in_channels=48, out_channels=72, kernel_size=3,stride=1, padding=2,dilation=2)
        self.relu7 = nn.ReLU(inplace=True)
        self.conv7_bn = nn.BatchNorm3d(72)
        self.conv8 = nn.Conv3d(in_channels=72, out_channels=72, kernel_size=3,stride=1, padding=2,dilation=2)
        self.relu8 = nn.ReLU(inplace=True)
        self.conv8_bn = nn.BatchNorm3d(72)
        self.pool4 = nn.AdaptiveAvgPool3d((1,1,1))
        self.dropout4 = nn.Dropout3d(conv_drop_rate)
        
        
        self.fc1 = nn.Linear(72, 2)
        
    def forward(self, x):
        x = self.conv1_bn(self.relu1(self.conv1(x)))
        x = self.dropout1(self.pool1(self.conv2_bn(self.relu2(self.conv2(x)))))
        
        x = self.conv3_bn(self.relu3(self.conv3(x)))
        x = self.dropout2(self.pool2(self.conv4_bn(self.relu4(self.conv4(x)))))
        
        x = self.conv5_bn(self.relu5(self.conv5(x)))
        x = self.dropout3(self.pool3(self.conv6_bn(self.relu6(self.conv6(x)))))
        
        x = self.conv7_bn(self.relu7(self.conv7(x)))
        x = self.conv8_bn(self.relu8(self.conv8(x)))
        
        x1 = self.dropout4(self.pool4(x))
#         x2 = self.dropout5(self.pool5(x))
        
        x1 = x1.view(-1, 72)
#         x2 = x2.view(-1, 72)
#         x = torch.cat((x1, x2), 1)
        x = x1
        #x = self.dropout5(self.fc1_bn(F.relu(self.fc1(x))))
        x = self.fc1(x)
        return x



In [13]:
# import torch.nn as nn
# import torch.nn.functional as F
# import torch

# class VGG_net(nn.Module):
#     def __init__(self,conv_drop_rate=0.10,linear_drop_rate=0.10):
#         super(VGG_net, self).__init__()
#         self.conv1 = nn.Conv3d(in_channels=1, out_channels=12, kernel_size=3,stride=1, padding=2,dilation=2)
#         self.conv1_bn = nn.BatchNorm3d(12)
#         self.conv2 = nn.Conv3d(in_channels=12, out_channels=12, kernel_size=3,stride=1,padding=2, dilation=2)
#         self.conv2_bn = nn.BatchNorm3d(12)
#         self.pool1 = nn.MaxPool3d(2, 2)
#         self.dropout1 = nn.Dropout3d(conv_drop_rate)
        
#         self.conv3 = nn.Conv3d(in_channels=12, out_channels=24, kernel_size=3,stride=1, padding=2,dilation=2)
#         self.conv3_bn = nn.BatchNorm3d(24)
#         self.conv4 = nn.Conv3d(in_channels=24, out_channels=24, kernel_size=3,stride=1, padding=2,dilation=2)
#         self.conv4_bn = nn.BatchNorm3d(24)
#         self.pool2 = nn.MaxPool3d(2, 2)
#         self.dropout2 = nn.Dropout3d(conv_drop_rate)
        
#         self.conv5 = nn.Conv3d(in_channels=24, out_channels=48, kernel_size=3,stride=1, padding=2,dilation=2)
#         self.conv5_bn = nn.BatchNorm3d(48)
#         self.conv6 = nn.Conv3d(in_channels=48, out_channels=48, kernel_size=3,stride=1, padding=2,dilation=2)
#         self.conv6_bn = nn.BatchNorm3d(48)
#         self.pool3 = nn.MaxPool3d(2, 2)
#         self.dropout3 = nn.Dropout3d(conv_drop_rate)
        
#         self.conv7 = nn.Conv3d(in_channels=48, out_channels=72, kernel_size=3,stride=1, padding=2,dilation=2)
#         self.conv7_bn = nn.BatchNorm3d(72)
#         self.conv8 = nn.Conv3d(in_channels=72, out_channels=72, kernel_size=3,stride=1, padding=2,dilation=2)
#         self.conv8_bn = nn.BatchNorm3d(72)
#         self.pool4 = nn.AdaptiveAvgPool3d((1,1,1))
#         self.dropout4 = nn.Dropout3d(conv_drop_rate)
        
        
#         self.fc1 = nn.Linear(72, 2)
        
#     def forward(self, x):
#         x = self.conv1_bn(F.relu(self.conv1(x)))
#         x = self.dropout1(self.pool1(self.conv2_bn(F.relu(self.conv2(x)))))
        
#         x = self.conv3_bn(F.relu(self.conv3(x)))
#         x = self.dropout2(self.pool2(self.conv4_bn(F.relu(self.conv4(x)))))
        
#         x = self.conv5_bn(F.relu(self.conv5(x)))
#         x = self.dropout3(self.pool3(self.conv6_bn(F.relu(self.conv6(x)))))
        
#         x = self.conv7_bn(F.relu(self.conv7(x)))
#         x = self.conv8_bn(F.relu(self.conv8(x)))
        
#         x1 = self.dropout4(self.pool4(x))
# #         x2 = self.dropout5(self.pool5(x))
        
#         x1 = x1.view(-1, 72)
# #         x2 = x2.view(-1, 72)
# #         x = torch.cat((x1, x2), 1)
#         x = x1
#         #x = self.dropout5(self.fc1_bn(F.relu(self.fc1(x))))
#         x = self.fc1(x)
#         return x


In [14]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    for i_batch, sample_batched in enumerate(train_loader):
        inputs, labels = sample_batched['image'], sample_batched['label']  
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        i_batch += 1
        if i_batch % 10 == 0:
            print("epoch {}, batch {}, current loss {}".format(epoch+1,i_batch,running_loss/10))
            running_loss = 0.0

def test(model, device, test_loader):
    model.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    
    true_predicted_labels = []
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels = sample_batched['image'], sample_batched['label']  
            inputs = inputs.to(device)
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            true_predicted_labels.append((labels.numpy(), predicted.cpu().numpy()))
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))
    return true_predicted_labels

def get_confusion_matrix(true_predicted_labels):
    cross_table = np.zeros([2,2])
    mut_to_nor = []
    nor_to_mul = []
    test_dic = true_predicted_labels
    for i in range(len(test_dic)):
        if test_dic[i][0] ==0 and test_dic[i][1] ==0:
            cross_table[0,0] += 1
        elif  test_dic[i][0] ==0 and test_dic[i][1] ==1:
            cross_table[0,1] += 1
            mut_to_nor.append(i)
        elif test_dic[i][0] ==1 and test_dic[i][1] ==0:
            cross_table[1,0] += 1
            nor_to_mul.append(i)
        elif test_dic[i][0] ==1 and test_dic[i][1] ==1:
            cross_table[1,1] += 1
    print(cross_table)
    print(mut_to_nor)
    print(nor_to_mul)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = VGG_net()
#net.apply(weight_init)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
net.to(device)
print("There are {} parameters in the model".format(count_parameters(net)))

num_epochs = 100
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([3.5,1.0]).to(device))

optimizer_base = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.00001)

optimizer = SWA(optimizer_base, swa_start=60, swa_freq=5)

print('choose SGD as optimizer')
#optimizer = optim.Adam(net.parameters(), lr=args.lr*10, weight_decay=0.00001)
#print('choose Adam as optimizer')

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=70, gamma=0.5)

for epoch in range(num_epochs):
    scheduler.step()
    
    Mouse_dataset = Mouse_sub_volumes(train_data)#transform=transforms.Compose([Flip()])
    dataloader = DataLoader(Mouse_dataset, batch_size=8, shuffle=True, num_workers=4, drop_last = True)
    train(net, device, dataloader, optimizer, criterion, epoch)
    if (epoch + 1) % 10 == 0:
        print('epoch {} train accuracy: '.format(epoch+1))
        train_Mouse_dataset = Mouse_sub_volumes(train_data)
        train_dataloader = DataLoader(train_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)
        train_dic = test(net, device, train_dataloader)
        get_confusion_matrix(train_dic)
        
        print("-------------------")
        print('epoch {} test accuracy: '.format(epoch+1))
        test_Mouse_dataset = Mouse_sub_volumes(test_data)
        test_dataloader = DataLoader(test_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)
        test_dic = test(net, device, test_dataloader)
        get_confusion_matrix(test_dic)
        
        torch.save(net.state_dict(), './model/mut_clas_2020_02_18_e{}_bv_body.pth'.format(epoch+1))
        
train_test_data = [train_data, test_data]
save_file = open('./model/mut_clas_2020_02_18_e{}_bv_body_data.pickle'.format(epoch+1),'wb')
pickle.dump(train_test_data, save_file)
save_file.close()

There are 355214 parameters in the model
choose SGD as optimizer
epoch 1, batch 10, current loss 0.6747636795043945
epoch 1, batch 20, current loss 0.690155291557312
epoch 1, batch 30, current loss 0.6850602984428406
epoch 1, batch 40, current loss 0.6396723717451096
epoch 1, batch 50, current loss 0.6886747300624847
epoch 2, batch 10, current loss 0.635322916507721
epoch 2, batch 20, current loss 0.6901465892791748
epoch 2, batch 30, current loss 0.6325404733419419
epoch 2, batch 40, current loss 0.610495176911354
epoch 2, batch 50, current loss 0.7106066405773163
epoch 3, batch 10, current loss 0.7380648553371429
epoch 3, batch 20, current loss 0.6472429931163788
epoch 3, batch 30, current loss 0.7002818435430527
epoch 3, batch 40, current loss 0.6520844459533691
epoch 3, batch 50, current loss 0.684912133216858
epoch 4, batch 10, current loss 0.6847851395606994
epoch 4, batch 20, current loss 0.6410349756479263
epoch 4, batch 30, current loss 0.7217817723751068
epoch 4, batch 40, cu

epoch 27, batch 10, current loss 0.19959915354847907
epoch 27, batch 20, current loss 0.2464704718440771
epoch 27, batch 30, current loss 0.23873214796185493
epoch 27, batch 40, current loss 0.2725895158946514
epoch 27, batch 50, current loss 0.0995297223329544
epoch 28, batch 10, current loss 0.2581790404394269
epoch 28, batch 20, current loss 0.3013050079345703
epoch 28, batch 30, current loss 0.28650456815958025
epoch 28, batch 40, current loss 0.3264158096164465
epoch 28, batch 50, current loss 0.2546819016337395
epoch 29, batch 10, current loss 0.21789034977555274
epoch 29, batch 20, current loss 0.15533548034727573
epoch 29, batch 30, current loss 0.21806127913296222
epoch 29, batch 40, current loss 0.16466109454631805
epoch 29, batch 50, current loss 0.17509342525154353
epoch 30, batch 10, current loss 0.14866357818245887
epoch 30, batch 20, current loss 0.4511845324188471
epoch 30, batch 30, current loss 0.22828017249703408
epoch 30, batch 40, current loss 0.19435512386262416
e

epoch 53, batch 50, current loss 0.05965827321633697
epoch 54, batch 10, current loss 0.12990458267740906
epoch 54, batch 20, current loss 0.025350341061130165
epoch 54, batch 30, current loss 0.14087775563821198
epoch 54, batch 40, current loss 0.20951480297371744
epoch 54, batch 50, current loss 0.12560680508613586
epoch 55, batch 10, current loss 0.05518680431414395
epoch 55, batch 20, current loss 0.12741815439076162
epoch 55, batch 30, current loss 0.031238413817482068
epoch 55, batch 40, current loss 0.08417800278402865
epoch 55, batch 50, current loss 0.08735472063999623
epoch 56, batch 10, current loss 0.049083476420491934
epoch 56, batch 20, current loss 0.024830990470945835
epoch 56, batch 30, current loss 0.03528400203213096
epoch 56, batch 40, current loss 0.06776834188494832
epoch 56, batch 50, current loss 0.05520205064676702
epoch 57, batch 10, current loss 0.05118629848875571
epoch 57, batch 20, current loss 0.02148262085975148
epoch 57, batch 30, current loss 0.1810556

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = VGG_net()
#net.apply(weight_init)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
net.to(device)
print("There are {} parameters in the model".format(count_parameters(net)))

In [ ]:
def test_with_probability(model, device, test_loader):
    model.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    
    true_predicted_labels = []
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels = sample_batched['image'], sample_batched['label']  
            inputs = inputs.to(device)
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            outputs = F.softmax(outputs)
            true_predicted_labels.append((labels.numpy(), predicted.cpu().numpy(), outputs.cpu().numpy()[0,0], outputs.cpu().numpy()[0,1]))
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))
    return true_predicted_labels

In [ ]:
#torch.save(net.state_dict(), './model/mut_clas_2019_01_15.pth')

In [ ]:
net.load_state_dict(torch.load('./model/mut_clas_2020_02_14_e120_bv_body.pth'))

with open('./model/mut_clas_2020_02_14_e120_bv_body_data.pickle', 'rb') as input_file:
    train_data, test_data = pickle.load(input_file)

In [ ]:
print('train accuracy: ')
Mouse_dataset = Mouse_sub_volumes(train_data)
train_dataloader = DataLoader(Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)

train_dic = test_with_probability(net, device, train_dataloader)

In [ ]:
print('test accuracy: ')
Mouse_dataset = Mouse_sub_volumes(test_data)
test_dataloader = DataLoader(Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)

test_dic = test_with_probability(net, device, test_dataloader)

In [ ]:
cross_table = np.zeros([2,2])
mut_to_nor = []
nor_to_mul = []

for i in range(len(test_dic)):
    if test_dic[i][0] ==0 and test_dic[i][1] ==0:
        cross_table[0,0] += 1
    elif  test_dic[i][0] ==0 and test_dic[i][1] ==1:
        cross_table[0,1] += 1
        mut_to_nor.append(i)
    elif test_dic[i][0] ==1 and test_dic[i][1] ==0:
        cross_table[1,0] += 1
        nor_to_mul.append(i)
    elif test_dic[i][0] ==1 and test_dic[i][1] ==1:
        cross_table[1,1] += 1
print(cross_table)
print(mut_to_nor)
print(nor_to_mul)

In [ ]:
def compute_saliency_maps(X, y, model):
    """
    Compute a class saliency map using the model for images X and labels y.

    Input:
    - X: Input images; Tensor of shape (N, 3, H, W)
    - y: Labels for X; LongTensor of shape (N,)
    - model: A pretrained CNN that will be used to compute the saliency map.

    Returns:
    - saliency: A Tensor of shape (N, H, W) giving the saliency maps for the input
    images.
    """
    # Make sure the model is in "test" mode
    model.eval()
    
    # Make input tensor require gradient
    X.requires_grad_()
    ##############################################################################
    # Perform a forward and backward pass through the model to compute the gradient 
    # of the correct class score with respect to each input image. You first want 
    # to compute the loss over the correct scores (we'll combine losses across a batch
    # by summing), and then compute the gradients with a backward pass.
    ##############################################################################
    scores = model(X)
    
    # Get the correct class computed scores.
    scores = scores.gather(1, y.view(-1, 1)).squeeze()  
    
    # Backward pass, need to supply initial gradients of same tensor shape as scores.
    scores.backward(torch.tensor(10.0).cuda(device))
    
    # Get gradient for image.
    saliency = X.grad.data
    
    # Convert from 3d to 1d.
    saliency = saliency.abs()
    saliency = saliency.squeeze()
    ##############################################################################
    return saliency

In [ ]:
test_Mouse_dataset = Mouse_sub_volumes(test_data)
test_dataloader = DataLoader(test_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)

for i_batch, sample_batched in enumerate(test_dataloader):
    inputs, labels = sample_batched['image'], sample_batched['label']  
    inputs = inputs.to(device)
    labels = labels.to(device)
    saliency = compute_saliency_maps(inputs, labels, net)
    
    max_value = torch.max(saliency)
    print(torch.max(saliency), torch.min(saliency))
    saliency[saliency >= (max_value*0.2)] = 100
    saliency[saliency < (max_value*0.2)] = 0
    saliency *= 0.01
    
    img_nft = nib.Nifti1Image(np.squeeze(inputs.cpu().detach().numpy()+1.0),np.eye(4))
    img_save_data_path = './saliency_map/img_label{}_{}.nii'.format(labels.cpu().numpy()[0], i_batch)
    nib.save(img_nft,img_save_data_path)
    
    print(i_batch, ' saliency num: ', np.sum(saliency.cpu().numpy()))
    saliency_nft = nib.Nifti1Image(np.squeeze(saliency.cpu().numpy()),np.eye(4))
    saliency_save_data_path = './saliency_map/salency_label{}_{}.nii'.format(labels.cpu().numpy()[0], i_batch)
    nib.save(saliency_nft,saliency_save_data_path)
    
# 5,6,8,9,10,11,12,33,27